In [1]:
import os, json
import jsonlines

DATA_DIR = "../data"
data_list = [
    "KoAlpaca/KoAlpaca_v1.1.jsonl",
    "KoChatGPT/kochatgpt_1_SFT.jsonl",
    "korquad-chat-v1/korquad-chat.json",
    "OIG-small-chip2-ko/oig-smallchip2-dedu.jsonl",
    "ShareGPT_DeepL/ko_alpaca_style_dataset.json",
    "kullm_v2/kullm-v2.jsonl",
    "KoAlpaca/ko_alpaca_data.json"]

In [2]:
from datasets import load_dataset

platypus = load_dataset("kyujinpy/KOpen-platypus")
# kocot = load_dataset("kyujinpy/KoCoT_2000")
ko_lima = load_dataset("taeshahn/ko-lima")
everylm = load_dataset("ziozzang/EverythingLM-data-V2-Ko")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/24926 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/kyujinpy___parquet/kyujinpy--KOpen-platypus-62cb0e3eebf44b4b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

No config specified, defaulting to: ko-lima/plain
Found cached dataset ko-lima (/root/.cache/huggingface/datasets/taeshahn___ko-lima/plain/1.0.0/dcae294ced838f0ead098a7936aeb3cf3a899649d6167c2219755aed4fbf8d10)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset json (/root/.cache/huggingface/datasets/ziozzang___json/ziozzang--EverythingLM-data-V2-Ko-8090253fcfb7eb38/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

### KoAlpaca v1.1 dataset

In [3]:
data_path = data_list[0]

koalpaca_v1_1 = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        koalpaca_v1_1.append(line)

len(koalpaca_v1_1)

21155

### KochatGPT SFT dataset

In [3]:
data_path = data_list[1]
with open(os.path.join(DATA_DIR, data_path), "r") as f:
    kochatgpt = json.load(f)

len(kochatgpt)

12000

### Korquad dataset

In [4]:
data_path = data_list[2]

korquad = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        korquad.append(line)

len(korquad)

9619

### OIG dataset

In [17]:
data_path = data_list[3]

oig = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        oig.append(line)

len(oig)

210282

### KULLM-v2 dataset

In [5]:
data_path = data_list[5]

kullm = []
with jsonlines.open(os.path.join(DATA_DIR, data_path), "r") as f:
    for line in f.iter():
        kullm.append(line)

len(kullm)

152627

### Processing Korquard dataset

In [5]:
convQA = []
L = len("<sys>")
for sample in korquad:#, total=len(korquad)):
    try:
        turns = sample["text"].split("\n")
        context = turns[0][L:].strip()
        dialog = []
        for turn in turns[1:]:
            role = turn[1:L-1]
            text = turn[L:].strip()
            dialog.append({"role": role, "text": text})
            
        assert dialog[0]["role"]=="usr" and len(dialog)%2==0
        
        inst_res = []
        for i in range(0, len(dialog), 2):
            user = dialog[i]["text"]
            if i==0:
                user = context.strip()+"\n\n"+user.strip()
            bot = dialog[i+1]["text"]
        
            inst_res.extend(
                [{"from": "human", "value": user.replace("<|bot|>", "어시스턴트")},
                 {"from": "bot", "value": bot.replace("<|user|>", "유저님"),}]
            )
        inst_res[1]["domain"] = "general"
                
        convQA.append(inst_res)
    except:
        pass

len(convQA)

9257

### 6-th Science

In [6]:
import random
from collections import Counter

with open("../data/science_edu/6th_grade_QA_science_filtered.json", "r") as fin:
    six_sci_qa = json.load(fin)

keywords = []
for sample in six_sci_qa:
    keywords.append(sample["keywords"].split(",")[0])

most_keywords = Counter(keywords).most_common(20)

sci_six_res = []
key_qa_map = {}
for sample in six_sci_qa:
    get_qa = True
    for keyword, _ in most_keywords:
        if keyword in sample["keywords"]:
            if keyword not in key_qa_map:
                key_qa_map[keyword] = [sample]
            else:
                key_qa_map[keyword].append(sample)
            get_qa = False
            break
    if get_qa:
        inst = f"{sample['question']}"
        res = sample['answer']
        sci_six_res.append({"instruction": inst, "output": res})
            

inst_form = "6학년 과학 {}에 대한 서술형 문제 {}개 만들어줘"

sci_six_gen = []
for key, qas in key_qa_map.items():
    index = 0
    while len(qas) > index:      
        problems = []
        cur_len = 0
        choice = random.randint(1,3)
        for k, s in enumerate(qas[index:index+choice]):
            Q, A = s["question"], s["answer"]
            problem = f"문제 {k+1}: {Q}\n답변: {A}"
            cur_len += len(problem)
            problems.append(problem)
            if cur_len > 500:
                choice = k+1
                break
        
        inst = inst_form.format(key, choice)
        index += choice
        sci_six_gen.append({"instruction": inst, "output": "\n\n".join(problems)})

print(sci_six_res[0])
print(sci_six_gen[0])

{'instruction': '꽃의 구조와 기능은 무엇일까요?', 'output': '꽃은 꽃받침, 꽃잎, 수술, 암술로 이루어져 있습니다. 꽃의 구조는 이 네 부분으로 구성되어 있습니다.'}
{'instruction': '6학년 과학 볼록 렌즈에 대한 서술형 문제 2개 만들어줘', 'output': '문제 1: 볼록 렌즈를 이용한 간이 프로젝터는 어떻게 만들까요?\n답변: 스마트 기기와 볼록 렌즈를 사용하여 스마트 기기의 영상을 스크린에 비추어 보는 간이 프로젝터를 만들 수 있습니다.\n\n문제 2: 볼록 렌즈를 이용한 간이 프로젝터 실험에서 스마트 기기의 영상은 어떻게 스크린에 보이나요?\n답변: 스마트 기기의 영상은 스크린에 뒤집혀서 보입니다.'}


### GSM8K_ko

In [7]:
import random, re

with open("../data/gsm8k/gsm8k_ko_main.json", "r") as fin:
    gsm8k = json.load(fin)

num_problem_gen = int(len(gsm8k) * 0.2)
random.shuffle(gsm8k)
gen_samples = gsm8k[:num_problem_gen]
gsm8k = gsm8k[num_problem_gen:]
insts = ["사칙연산을 연습할 수 있도록 다단계 추론이 필요한 단답형 수학 문제 {}개 작성해줘.",
         "사칙연산을 연습하고자 하는데, 다단계 추론을 필요로 하는 단답형 수학 문제를 {}개 만들어주세요.",
         "다단계 추론이 필요한 사칙연산 연습용 단답형 수학 문제를 {}문제 작성 부탁드립니다.",
         "단답형 수학 문제 {}개를 만들어주세요. 이 문제들은 사칙연산을 연습하면서 다단계 추론을 요구합니다."]
gsm8k_gen = []
index = 0
while num_problem_gen > index:
    choice = random.randint(1,3)
    cur_len = 0
    problems = []
    for k, s in enumerate(gen_samples[index:index+choice]):
        Q, A = s["question"], s["answer"]
        A = re.sub(r"\\equ{([0-9-+*/=.]+)}", r"\1", A)
        A = A.replace(" -> ", "\n")
        problem = f"문제{k+1}: {Q}\n해설: {A}"
        cur_len += len(problem)
        problems.append(problem)
        if cur_len > 500:
            choice = k+1
            break
    
    inst = random.choice(insts)
    inst = inst.format(choice)
    index += choice
    gsm8k_gen.append({"instruction": inst, "output": "\n\n".join(problems)})

gsm8k_solve = []
for sample in gsm8k:
    gsm8k_solve.append({
        "instruction": sample["question"],
        "output": sample["answer"]
    })
        
print(len(gsm8k_gen), len(gsm8k_solve))

865 5913


In [8]:
import random

with open("../data/gsm8k/gsm8k_ko_socratic.json", "r") as fin:
    gsm8k = json.load(fin)

prompt = ["소크라테스식 문답으로 풀어줘.", "다단계 추론을 거쳐 정답을 추론해줘."]

gsm8k_socra = []
for sample in gsm8k:
    Q, A = s["question"], s["answer"]
    A = re.sub(r"\\equ{([0-9-+*/=.]+)}", r"\1", A)
    A = A.replace(" -> ", "\n")
    gsm8k_socra.append({
        "instruction": Q+random.choice(prompt),
        "output": sample["answer"]
    })
        
print(len(gsm8k_socra))

7374


### AI Hub Book summarization

In [9]:
import os, json
from glob import glob
from tqdm import tqdm

json_files = glob("../data/AIhub_book_summary/*.json")

cand_prompts = [
    "이 문단을 한 문장에서 세 문장 정도로 간단히 요약해줘.",
    "이 문단을 한 문장에서 세 문장으로 간략하게 요약해 주세요.",
    "위 글을 간략하게 핵심 요약해줘.",
    "이 글을 한 문장에서 세 문장으로 간단하게 요약해주세요."
]

ins1_format = "{passage}\n\n이 문단을 한 문장에서 세 문장 정도로 간단히 요약해줘."
res1_format = "{summary}"
ins2_format = "이 문단이 포함될 문서의 제목을 지어줘."
res2_format = "제목: {doc_name}"

target_kdc = ["보건의료", "체육", "보육·가족및여성", "한국문학", "한국어", "문학", "문화재", "역사", "음악", "심리학"]
book_summary = []
kdcs = []
pretrained_data = []
for path in (json_files):
    with open(path, "r") as f:
        json_data = json.load(f)
    data = json_data["metadata"]
    kdcs.append(data["kdc_label"])
        
    if "교육" in data["kdc_label"] or data["kdc_label"] in target_kdc:
        pretrained_data.append(f"{data['kdc_label']} {data['doc_type']} \'{data['doc_name']}\'에서 발췌:\n\n"+json_data["passage"])
        data["passage"] = json_data["passage"].strip()
        data["summary"] = json_data["summary"].strip()
        prompt = random.choice(cand_prompts)
        ins1_format = "{passage}\n\n"+prompt
        ins1 = ins1_format.format_map(data)
        res1 = res1_format.format_map(data)
        ins2 = ins2_format.format_map(data)
        res2 = res2_format.format_map(data)
        
        book_summary.append([
            {"from":"human", "value":ins1},
            {"from":"bot", "value":res1, "domain":"general"},
            {"from":"human", "value":ins2},
            {"from":"bot", "value":res2},
        ])

In [10]:
len(book_summary)

32523

In [10]:
# print(koalpaca_v1_0[0])
print(koalpaca_v1_1[0])
# print(kochatgpt[0])
print(convQA[0])
# print(oig[0])
# print(sharegpt[0])
# print(kullm[-11])

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?', 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}
[{'from': 'human', 'value': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 

In [11]:
print(len(convQA)+len(koalpaca_v1_1))
print(len(book_summary)) # 1000
print(len(gsm8k_solve))
print(len(gsm8k_gen))
# print(len(gsm8k_socra))
# print(len(sci_six_gen))
print(len(sci_six_res))

30412
32523
5913
852
778


In [6]:
import re, json
from tqdm import tqdm
import random

matching_number = {
    "①": "(1)",
    "②": "(2)",
    "③": "(3)",
    "④": "(4)",
    "⑤": "(5)"
}

def to_dialog(data, inst_key, out_key, domain, inp_key="input", preprossing=True):
    dialog = []
    for sample in tqdm(data, total=len(data)):
        if preprossing:
            inst = re.sub(r"\?(\w)", r"?\n\1", sample[inst_key])
            output = re.sub(r"[.](\d+)[.]", r".\n\1.", sample[out_key])
        else:
            inst = sample[inst_key]
            output = sample[out_key]
            
        if inp_key in sample and sample[inp_key]:
            inst = sample[inp_key].strip()+"\n\n"+inst
            
        if domain!="education" and ("OpenAI" in output or "코드" in inst or re.match(r"[가-힣]", output) is None):
            continue
        
        if domain=="education":
            for k, v in matching_number.items():
                inst = inst.replace(k, v)
                output = output.replace(k, v)
                output = re.sub(r"[ ]+\n", "\n", output)
    
        dialog.append(
            [{"from": "human", "value": inst},
             {"from": "bot", "value": output, "domain":domain}]
        )
    print(f"Get {len(dialog)} dialogs")
    return dialog

### Platypus

In [14]:
# print("Unifying except for Korquad dataset")
unified_data = []
# # koalpacav1.1
unified_data.extend(to_dialog(koalpaca_v1_1, "instruction", "output", "general"))
# Korquad
unified_data.extend(convQA)

# Book summary
# _book_summary = random.sample(book_summary, 3000)
# unified_data.extend(_book_summary)

with open("../data/extra_edu_data.json", "r", encoding="utf-8") as f:
    extra = json.load(f)
unified_data.extend(to_dialog(extra, "instruction", "output", "education"))

print(len(unified_data))

# artificial_data = []
# GSM8K
# artificial_data.extend(to_dialog(gsm8k_solve, "instruction", "output", "general"))
# artificial_data.extend(to_dialog(gsm8k_gen, "instruction", "output", "general"))
# artificial_data.extend(to_dialog(gsm8k_socra, "instruction", "output", "general"))
# Sci 6 grade
# artificial_data.extend(to_dialog(sci_six_gen, "instruction", "output", "general"))
# artificial_data.extend(to_dialog(sci_six_res, "instruction", "output", "general"))
# artificial_data = random.sample(artificial_data, len(unified_data))

# unified_data.extend(artificial_data)

# Dialog
with open("../data/web_crawled/naver2dialog.json", "r") as f:
    conv_data = json.load(f)
    
unified_data.extend(conv_data)

# kochatgpt
# unified_data.extend(to_dialog(kochatgpt, "prompt", "completion", "general"))
# OIG
# unified_data.extend(to_dialog(oig, "user_translated", "chip2_translated", "general"))

100%|██████████| 21155/21155 [00:00<00:00, 73438.30it/s]


Get 18483 dialogs


100%|██████████| 14/14 [00:00<00:00, 12868.78it/s]

Get 14 dialogs
27754


In [15]:
len(unified_data)

29644

In [16]:
# KULLM-v2
# processed_kullm = []
# for sample in tqdm(kullm, total=len(kullm)):
#     if sample["input"]:
#         inst = sample["input"].strip()+"\n\n"+sample["instruction"].strip()
#     else:
#         inst = sample["instruction"].strip()
        
#     if len(inst) <= 15:
#         continue
    
#     output = re.sub(r"[.](\d+)[.]", r".\r\n\1.", sample["output"])
#     if "OpenAI" in output or "코드" in inst or re.match(r"[가-힣]", output) is None:
#         continue
        
#     processed_kullm.append(
#         [{"from": "human", "value": inst}, 
#          {"from": "bot", "value": output, "domain": "general"}]
#     )

# unified_data.extend(random.sample(processed_kullm, 30000))

general_data = []
general_data.extend(to_dialog(platypus["train"], "instruction", "output", domain="general", preprossing=False))
general_data.extend(to_dialog(everylm["train"], "instruction", "output", domain="general", preprossing=False))
# general_data.extend(to_dialog(kocot["train"], "source", "rationale", domain="general", preprossing=False))

kolima_processed = []
for sample in tqdm(ko_lima["train"], total=len(ko_lima["train"])):
    conv = sample["conversations"]
    
    turns = []
    for t in range(0, len(conv), 2):
        try:
            turns.extend(
                [{"from": "human", "value": conv[t]}, 
                {"from": "bot", "value": conv[t+1]}]
            )
        except Exception as e:
            print(e)
            break
        if t==0:
            turns[1]["domain"] = "general"
    
    if turns:
        kolima_processed.append(turns.copy())
        
general_data.extend(kolima_processed)

print(len(general_data))

  0%|          | 0/24926 [00:00<?, ?it/s]

100%|██████████| 24926/24926 [00:01<00:00, 16531.92it/s]


Get 12306 dialogs


100%|██████████| 991/991 [00:00<00:00, 16501.40it/s]


Get 774 dialogs


100%|██████████| 1030/1030 [00:00<00:00, 18343.58it/s]

list index out of range
14110


In [17]:
unified_data.extend(general_data)
len(unified_data)

43754

In [14]:
with open("../data/inst_data_wo_edu.json", "w") as fout:
    json.dump(unified_data, fout, ensure_ascii=False, indent=2)

In [18]:
import random, os

PB_DIR = "../data/problem_bank"

## 객관식 문제
with open(os.path.join(PB_DIR, "context_5_choices.json"), "r") as f:
    non_pb_data = json.load(f)
    
pb_data = []

for sample in non_pb_data:
    problem, bogi, answer, explain = sample["response"].split("\n")
    bogi = re.sub(r"([^①②③④⑤]),( )?([①②③④⑤])", r"\1\n\3", bogi)
    sample["response"] = f"{problem}\n{bogi}\n{explain}\n{answer}"
    sample["response"] = sample["response"].replace("보기: ", "보기:\n")
    pb_data.append(sample)

choice_dir = f"{PB_DIR}/5_choices"
essay_dir = f"{PB_DIR}/essay"
short_ans_dir = f"{PB_DIR}/short_answer"
choices5 = os.listdir(choice_dir)
essay = os.listdir(essay_dir)
short_ans = os.listdir(short_ans_dir)

choice_data = []
for filename in choices5:
    with open(os.path.join(choice_dir, filename), "r") as f:
        choice_data.extend(json.load(f))

choices_multiple = {}
for sample in choice_data:
    if "대분수" in sample["Assistant"] or "가분수" in sample["Assistant"]:
        continue
    if sample["User"] in choices_multiple and not sample["Assistant"] in choices_multiple[sample["User"]]:
        choices_multiple[sample["User"]].append(sample["Assistant"])
    else:
        choices_multiple[sample["User"]] = [sample["Assistant"]]

NUM_DICT = {
    1: ["하나", "한 문제", "한 개", "1개", "1 문항"],
    2: ["둘", "두 문제", "두 개", "2개", "2 문항"],
    3: ["셋", "세 문제", "세 개", "3개", "3 문항"],
    4: ["넷", "네 문제", "네 개", "4개", "4 문항"]
}

CHOICE_DICT = {
    0: "에 대한 오지선다 문제",
    1: "에 대한 5지선택 문제",
    2: " 주제로 5개 보기를 가진 객관식 문제",
    3: "에 대해 보기 5개가 주어지는 객관식 문제",
    4: " 단원에 대한 객관식 문항"
}

choices_inst_data = []
for inst, prbs in choices_multiple.items():
    index = 0
    while index < len(prbs):
        num_prb = random.randint(1, 7)
        num_prb = min(num_prb, len(prbs)-index)
        if num_prb > 4:
            num_prb = 1
        num_txt = random.choice(NUM_DICT[num_prb])
        changed_inst = inst.replace("하나 만들어줘.", f"{num_txt} 만들어줘." if random.randint(0,1) else f"{num_txt} 제출해줘.")
        inst_rand = CHOICE_DICT[random.randint(0,4)]
        changed_inst = changed_inst.replace(" 5지선택문제", inst_rand).replace("None학년 None학기 ", "")

        gen_problem = prbs[index:index+num_prb]
        refined_problem = []
        idx = 1
        for prob in gen_problem:
            prob = prob.replace("문제:", f"문제{idx}:")
            splited = prob.split("\n")
            if splited[1][:len("보기:")]=="보기:":
                splited[1] = re.sub(r"([^①②③④⑤]),( )?([①②③④⑤])", r"\1\n\3", splited[1])
                splited[1] = splited[1].replace("보기: ", "보기:\n")
                
            correct = splited[-2]
            if "[해설]" in correct:
                cor_splited = correct.split("[해설]")
                correct = cor_splited[0]
                if len(cor_splited[1]) > 3:
                    solution = "해설: "+cor_splited[1]
                else:
                    solution = splited[-1]
            else:
                solution = splited[-1]
            solution = solution.replace("]]", "")
            solution = solution.replace("해설: .", "해설: ")
            if len(solution) < len("해설: ")+2:
                continue
            
            problem = splited[:-2]
            problem.extend([solution, correct])
            refined_problem.append("\n".join(problem))
            idx += 1

        gen_problem = "\n\n".join(refined_problem)
        gen_problem = re.sub(r"([①②③④⑤])([가-힣0-9])", r"\1 \2", gen_problem)
        gen_problem = re.sub(r"([가-힣0-9.,?!])([①②③④⑤])", r"\1 \2", gen_problem)
        gen_problem = re.sub(r"[ ]{2,}", r" ", gen_problem)
        gen_problem = re.sub(r"([^\s])따라서", r"\1 따라서", gen_problem)
        gen_problem = re.sub(r"([가-힣])[.]([가-힣])", r"\1. \2", gen_problem)
        gen_problem = re.sub(r"([가-힣!.?])[\s]?[·•-] ([가-힣])", r"\1\n- \2", gen_problem)
        gen_problem = re.sub(r"([^\s])<보기>", r"\1\n<보기>", gen_problem)
        gen_problem = gen_problem.replace("曹曹曹曹曹曹曹曹", "A")
        gen_problem = gen_problem.replace("曹", "")
        
        if not gen_problem:
            index += num_prb
            continue
        
        if num_prb==1 and random.randint(0,1):
            splited = gen_problem.split("\n")
            try:
                solution = splited[-2]
                correct = splited[-1]
            except:
                print(gen_problem)
                print(splited)
            output = f"{solution}\n{correct}"
            problem = "\n".join(splited[:-2])
            
            if "<보기>" in problem and len(problem.split("<보기>")) <= 2 and not "<보기>\nㄱ" in problem:
                output = "주어져야 할 보기가 제공되지 않았습니다. 문제를 풀기 위해서는 더 많은 정보가 필요합니다. 적절한 정보를 제공해주세요."
            elif ("주어진" in problem or "빈 칸" in problem or "빈칸" in problem) and not "__" in problem and not "괄호" in problem and not "주어진 힘" in problem and not "영어로" in problem:
                output = "주어져야 할 정보가 제공되지 않았습니다. 문제를 풀기 위해서는 더 많은 정보가 필요합니다. 적절한 정보를 제공해주세요."
                
            choices_inst_data.append({"user": problem.replace("문제1:","문제:"), "response": output})
        else:
            choices_inst_data.append({"user": changed_inst, "response": gen_problem})
            
        index += num_prb
        
## 에세이
essay_data = []
for filename in essay:
    with open(os.path.join(essay_dir, filename), "r") as f:
        essay_data.extend(json.load(f))
        
essay_multiple = {}
for sample in essay_data:
    if sample["Q"] in essay_multiple and not sample["A"] in essay_multiple[sample["Q"]]:
        essay_multiple[sample["Q"]].append(sample["A"])
    else:
        essay_multiple[sample["Q"]] = [sample["A"]]
        
essay_inst_data = []
for inst, prbs in essay_multiple.items():
    index = 0
    while index < len(prbs):
        num_prb = random.randint(1, 6)
        num_prb = min(num_prb, len(prbs)-index)
        if num_prb > 3:
            num_prb = 1
        num_txt = random.choice(NUM_DICT[num_prb])
        changed_inst = inst.replace("하나 생성해줘", f"{num_txt} 만들어줘." if random.randint(0,1) else f"{num_txt} 제출해줘.")
        changed_inst = changed_inst.replace(", 논술형 문제", "에 관한 논술형 문제").replace("None학년 None학기 ", "")

        gen_problem = prbs[index:index+num_prb]
        refined_problem = []
        idx = 1
        for prob in gen_problem:
            prob = prob.replace("문제 :", f"문제{idx}:")
            correct, solution = prob.split(", 해설 : ")
            problem, correct = correct.split(", 정답 : ")
            
            if not solution.strip():
                continue
            
            if correct in solution or correct=="풀이 참조":
                output = f"정답 및 해설: {solution}"
            elif not solution:
                output = f"정답 및 해설: {correct}"
            else:
                output = f"해설: {solution}\n정답: {correct}"
                
            output = re.sub(r"[ ]{2,}", r" ", output)
            output = re.sub(r"([0-9])([가-힣])", r"\1 \2", output)
            problem = re.sub(r"([가-힣a-z])[.]([가-힣A-Z])", r"\1. \2", problem)
            problem = re.sub(r"([^\s])([ㄱㄴㄷㄹㅁㅂ])", r"\1\n\2", problem)
            problem = re.sub(r"[.]<보기>", ".\n<보기>", problem)
            output = re.sub(r"([가-힣a-z])[.]([가-힣A-Z])", r"\1. \2", output)
            output = re.sub(r"([가-힣!.?])[\s]?[·•-] ([가-힣])", r"\1\n- \2", output)
            
            refined_problem.append((problem, output))
            idx += 1
            
        if not refined_problem:
            index += num_prb
            continue
        
        if num_prb==1 and random.randint(0,1):
            problem, output = refined_problem[0]
            if "<보기>" in problem and len(problem.split("<보기>")) <= 2 and not "<보기>\nㄱ" in problem:
                output = "주어져야 할 보기가 제공되지 않았습니다. 문제를 풀기 위해서는 더 많은 정보가 필요합니다. 적절한 정보를 제공해주세요."
            elif ("주어진" in problem or "빈 칸" in problem or "빈칸" in problem) and not "__" in problem and not "괄호" in problem and not "주어진 힘" in problem and not "영어로" in problem:
                output = "주어져야 할 정보가 제공되지 않았습니다. 문제를 풀기 위해서는 더 많은 정보가 필요합니다. 적절한 정보를 제공해주세요."
            essay_inst_data.append({"user": problem.replace("문제1:","문제:"), "response": output})
        else:
            gen_problem = "\n\n".join([f"{problem}\n{output}" for problem, output in refined_problem])
            essay_inst_data.append({"user": changed_inst, "response": gen_problem})
            
        index += num_prb
        
        
## 단답형
short_ans_data = []
for filename in short_ans:
    with open(os.path.join(short_ans_dir, filename), "r") as f:
        short_ans_data.extend(json.load(f))
        
short_ans_multiple = {}
for sample in short_ans_data:
    if sample["Q"] in short_ans_multiple and not sample["A"] in short_ans_multiple[sample["Q"]]:
        short_ans_multiple[sample["Q"]].append(sample["A"])
    else:
        short_ans_multiple[sample["Q"]] = [sample["A"]]
        
short_ans_inst_data = []
for inst, prbs in short_ans_multiple.items():
    if "계산해 보세요." in inst:
        continue
    index = 0
    while index < len(prbs):
        num_prb = random.randint(1, 7)
        num_prb = min(num_prb, len(prbs)-index)
                
        if num_prb > 4:
            num_prb = 1
            
        if "[Exam4u]과목" in inst or "[출제원리를 찾아서]" in inst:
            num_prob = 1
            
        num_txt = random.choice(NUM_DICT[num_prb])
        changed_inst = inst.replace("하나 생성해줘.", f"{num_txt} 만들어줘." if random.randint(0,1) else f"{num_txt} 제출해줘.")
        changed_inst = changed_inst.replace(" 단답유순형", "에 관한 단답형")
        changed_inst = changed_inst.replace(" 단답무순형", "에 관한 단답형").replace("None학년 None학기 ", "")

        gen_problem = prbs[index:index+num_prb]
        refined_problem = []
        idx = 1
        for prob in gen_problem:
            prob = prob.replace("문제 :", f"문제{idx}:")
            correct, solution = prob.split(", 해설 : ")
            problem, correct = correct.split(", 정답 : ")
            
            if not solution.strip():
                continue
            
            if correct in solution or correct=="풀이 참조":
                output = f"정답 및 해설: {solution}"
            elif not solution:
                output = f"정답 및 해설: {correct}"
            else:
                output = f"해설: {solution}\n정답: {correct}"
                
            output = re.sub(r"[ ]{2,}", r" ", output)
            output = re.sub(r"([0-9])([가-힣])", r"\1 \2", output)
            problem = re.sub(r"([가-힣a-z])[.]([가-힣A-Z])", r"\1. \2", problem)
            problem = re.sub(r"([^\s])([ㄱㄴㄷㄹㅁㅂ])", r"\1\n\2", problem)
            problem = re.sub(r"[.]( )?<보기>", ".\n<보기>", problem)
            output = re.sub(r"([가-힣!.?])[\s]?[·•-] ([가-힣])", r"\1\n- \2", output)
            output = re.sub(r"([가-힣a-z])[.]([가-힣A-Z])", r"\1. \2", output)
            
            refined_problem.append((problem, output))
            idx += 1
            
        if not refined_problem:
            index += num_prb
            continue
        
        if (num_prb==1 and random.randint(0,1)) or "[Exam4u]과목" in inst or "[출제원리를 찾아서]" in inst:
            problem, output = refined_problem[0]
            
            if "<보기>" in problem and len(problem.split("<보기>")) <= 2 and not "<보기>\nㄱ" in problem:
                output = "주어져야 할 보기가 제공되지 않았습니다. 문제를 풀기 위해서는 더 많은 정보가 필요합니다. 적절한 정보를 제공해주세요."
            elif ("주어진" in problem or "빈 칸" in problem or "빈칸" in problem or "빈 곳" in problem) and not "__" in problem and not "괄호" in problem and not "주어진 힘" in problem and not "영어로" in problem:
                output = "주어져야 할 정보가 제공되지 않았습니다. 문제를 풀기 위해서는 더 많은 정보가 필요합니다. 적절한 정보를 제공해주세요."
                
            short_ans_inst_data.append({"user": problem.replace("문제1:","문제:").strip(), "response": output.strip()})
        else:
            gen_problem = "\n\n".join([f"{problem}\n{output}" for problem, output in refined_problem])
            short_ans_inst_data.append({"user": changed_inst.strip(), "response": gen_problem.strip()})
            
        index += num_prb

In [19]:
pb_data.extend(choices_inst_data)
pb_data.extend(essay_inst_data)
pb_data.extend(short_ans_inst_data)

print(len(pb_data))

with open("../data/processed_math_eng_data.json", "r") as fin:
    add_data = json.load(fin)
    
pb_data.extend(add_data)

print(len(pb_data))

with open("../data/pb_data.json", "w") as fout:
    json.dump(pb_data, fout, indent=2, ensure_ascii=False)

9892
14310


In [20]:
# pb_data = random.sample(pb_data, k=8192)
# edu_data = []
unified_data.extend(to_dialog(pb_data, "user", "response", "education"))
len(unified_data)

  0%|          | 0/14310 [00:00<?, ?it/s]

100%|██████████| 14310/14310 [00:00<00:00, 17144.30it/s]

Get 14310 dialogs


58064

### AI hub 주제별 일상 대화 데이터

In [24]:
from glob import glob
import json

chitchat_list = glob("../data/AIhub_chitchat/*.json")
chitchat_data = []

for path in chitchat_list:
    try:
        with open(path, "r") as fin:
            chitchat_data.append(json.load(fin))
    except Exception:
        print(path)
        continue

len(chitchat_data)

../data/AIhub_chitchat/KAKAO_1648_13.json


98651

In [25]:
from tqdm import tqdm
import re

dialogs = []
for data in chitchat_data:
    data = data["info"][0]
    domain = data["annotations"]["speaker_type"]
    if data["annotations"]["speaker_type"]!="1:1":
        continue
    
    data = data["annotations"]
    lines = data["lines"]
    
    if len(lines) < 2 or not data["lines"][0]["norm_text"]:
        continue
    
    if len(set([line["speaker"]["id"] for line in lines]))!=2:
        continue
       
    user_info = lines[0]["speaker"]
    u_id = user_info["id"]
    u_gender = user_info["sex"]
    u_age = user_info["age"]
    
    prev_id = u_id
    dialog = []
    user = []
    bot = []
    for i in range(len(lines)):
        cur_id = lines[i]["speaker"]["id"]
        if prev_id!=cur_id and prev_id!=u_id:
            response = " ".join(bot).strip()
            response = re.sub("키키", "ㅋㅋ", response)
            dialog.extend(
                [{"from": "human", "value": " ".join(user).strip()},
                 {"persona": {"age": a_age, "gender": a_gender, "domain": "일상 대화"}, "from": "bot", "value": response}]
            )
            user = []
            bot = []

        if cur_id!=u_id:
            a_gender = lines[i]["speaker"]["sex"]
            a_age = lines[i]["speaker"]["age"]
            bot.append(lines[i]["norm_text"])
        else:
            user.append(lines[i]["norm_text"])
            
        prev_id = cur_id
    
    if dialog:
        dialogs.append(dialog.copy())
    
print(len(dialogs))

unified_data.extend(dialogs)

70613


In [20]:
len(unified_data)

79830

In [15]:
import json

with open("../data/web_crawled/naver2dialog.json", "r") as fin:
    data = json.load(fin)
    
dialogs = []
for dial in data:
    check = True
    for sample in dial:
        if sample["from"]=="bot" and "?" in sample["value"]:
            check = False
            break
    if check:
        dialogs.append(dial)

print(len(dialogs))
len(data)

1890


1890

In [21]:
import random

with open("../data/unified_DAPT_data.json", "r") as fin:
    dapt = json.load(fin)

formats = ["{area}에 관련해서 {topic}을 주제로 수업자료를 만들어줘.",
           "{area}, {topic}에 관련한 교육 자료를 개발해줘.",
           "{area} 과목 내 {topic}를 주제로 다루는 교재를 만들어봐.",
           "{topic} 관련 {area} 수업에서 사용할 자료를 제작해줘.",
           "{area} 과목과 관련지어 {topic}를 포함한 강의 자료를 작성해줘.",
           "{area}의 주제 '{topic}'로 구성한 강의 내용을 준비해줘.",
           "{topic}를 주제로 한 {area} 관련 수업 자료를 만들어줘.",
           "{topic}에 초점을 맞춘 {area} 교육용 자료를 써줘.",
           "{area}에서의 {topic}에 대한 교육용 자료를 만들어주세요.",
           "{topic} 내용을 담은 {area} 교육 자료를 구성해줘."]

format_kor = "{topic}에 대해 올바르게 알 수 있도록 교훈을 줄 수 있는 이야기나 사례를 작성해줘."
format_per = "{topic}의 생애와 업적 등을 소개하는 글을 써줘."
format_e = "'{topic}' 라는 주제로 어린이를 위한 흥미로운 수업 자료를 작성해줘."
format_so = "학생들을 위해 '{topic}' 라는 주제를 가지고 흥미롭게 설명하는 글을 작성해줘."

pairs = []
additional = []
for sample in dapt:
    area = sample.split(" ")[0]
    if area[-1]==":" and not area[:-1] in ["예체능", "성장백과"]:
        try:
            splited = sample.split("\n\n")
            area_topic = splited[0].split(": ")
            area = area_topic[0]
            if len(area_topic) > 2:
                topic = f"{area_topic[1].strip()} - {area_topic[2].strip()}"
            else:
                topic = area_topic[1]
            if (area, topic) in pairs:
                continue
            if topic=="지도로 보는 프롤로그":
                continue
            content = splited[1]
            if type(content) is list:
                content = "\n\n".join(content)
            content = re.sub(r"([^가-힣]) : ([가-힣])", r"\1\n\2", content).strip()
            content = re.sub(r"”([가-힣 ])", r"”\n\1", content)
            content = re.sub(r"([가-힣.?! ”])“", r"\1\n“", content)
            content = re.sub(r"([^#\n])## ([가-힣])", r"\1\n\n## \2", content)
            content = re.sub(r"([가-힣!.?])[\s]?[·•-] ([가-힣])", r"\1\n- \2", content)
            content = re.sub(r"([가-힣]+: [가-힣\s?!.]+[?!.]) ([가-힣]+:)", r"\1\n\2", content)
            content = re.sub(r"([가-힣]+: [가-힣\s?!.]+[?!.]) ([가-힣]+:)", r"\1\n\2", content)
            content = re.sub(r"([가-힣])([.?!])( )?([(])?([\d])([.)])( )?([가-힣])", r"\1\2\n\4\5\6 \8", content)
            content = content.replace("#풀이\n", "")
            content = content.replace("\n ", "\n")
            content = content.replace(" \n", "\n")
            content = content.replace("##\n", "")
             
        except Exception as e:
            print(e, splited)
            continue
        if area=="국어":
            form = format_kor
        elif area=="인물탐구":
            form = format_per
        elif area=="지식e":
            form = format_e
        elif area=="사회":
            form = format_so
        else:
            form = random.choice(formats)
        
        instruction = form.format_map({"area":area, "topic":topic.replace("\n"," ").replace("  ", " ").strip()})
        instruction = re.sub(r"[\?]([^\s])", r"? \1", instruction)
        additional.append([{"from": "human", "value": instruction}, {"from":"bot", "value": content, "domain": "education"}])
        pairs.append((area, topic))

In [22]:
len(additional)

10971

In [23]:
# additional = random.sample(additional, k=8192)
unified_data.extend(additional)

In [9]:
# with open("../data/extra_edu_data.json", "r", encoding="utf-8") as f:
#     extra = json.load(f)
# edu_data.extend(to_dialog(extra, "instruction", "output", "education"))

# # Dialog
# with open("../data/web_crawled/naver2dialog.json", "r") as f:
#     conv_data = json.load(f)
    
# edu_data.extend(conv_data)

100%|██████████| 14/14 [00:00<00:00, 7378.77it/s]

Get 14 dialogs


In [10]:
import json

with open("../data/inst_edu_third.json", "w", encoding="utf-8") as f:
    json.dump(unified_data, f, ensure_ascii=False, indent=2) # ensure_ascii로 한글이 깨지지 않게 저장

In [2]:
import json, re

with open("../data/unified_DAPT_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    
new_data = []

for sample in data:
    sample = re.sub(r"([가-힣.?!]) - ([가-힣0-9])", r"\1\n- \2", sample) 
    new_data.append(sample)
    
with open("../data/unified_DAPT_data.json", "w", encoding="utf-8") as f:
    json.dump(new_data, f, ensure_ascii=False, indent=2) # ensure_ascii로 한글이 깨지지 않게 저장

In [124]:
a = "선생님: 세다 토론이라고 들어 본 적 있나요? 희문: 세다? 그건 상대보다 무엇을 잘할 때 세다라고 하는 말 아니에요."
# re.sub(r"([^\s])([①②③④⑤])", r"\1 \2", a)
re.sub(r"([가-힣]+: [가-힣\s?!.]+[?!.]) ([가-힣]+:)", r"\1\n\2", a)

'선생님: 세다 토론이라고 들어 본 적 있나요?\n희문: 세다? 그건 상대보다 무엇을 잘할 때 세다라고 하는 말 아니에요.'

In [1]:
import torch

data = torch.load("../data/inst_edu_v5.pt")

In [5]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained("../experiments/poly12.8b-DAPT")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tokenizer.decode(data["input_ids"][1].tolist())

'호기심 많은 유저와 어시스턴트 간의 대화. 어시스턴트는 유저의 질문이나 지시에 도움이 되고 상세하며 정중한 답변을 합니다.\n\n### 질문: 스웨터의 유래는 어디에서 시작되었나요?\n\n### 답변: 스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까지 이어지고 있습니다.<|endoftext|>'

In [37]:
import transformers



loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [38]:
tokenizer.decode(data["input_ids"][30000])

'호기심 많은 유저와 인공지능 어시스턴트 간의 대화. 어시스턴트는 유저의 질문이나 지시에 도움이 되고 상세하며 정중한 답변을 합니다. USER: 이재명 성남시장은 (6·4 지방선거의 성남시장 후보 자격으로서) 2014년 5월 15일 자신의 트위터에 “변희재 법적조치 중”이라며 “명예훼손, 모욕, 선거법 위반 혐의로 고발했습니다”라고 밝혔다. 이어, “끝까지 책임을 물어서 ‘행동에는 책임이 따른다’는 평범한 진리를 깨우쳐 주겠습니다”라며 “민사 손해배상소송도 제기합니다”라고 전했다. 이 후보측은 \'명예훼손 및 모욕, 공직선거법 위반 등의 혐의로 변 대표를 처벌해달라\'며 서울지방경찰청 영등포경찰서에 제출한 고소장에서 "피고는 피고소인을 \'종북\' 또는 \'종북세력에 기생하는 종북거머리\'라고 지칭하고 \'북한의 지령을 받고 움직이는 자들은 종북이 아니라 간첩\'이라는 내용의 허위사실을 지난해 1월부터 최근까지 총 13차례에 걸쳐 자신의 SNS에 올려 이재명의 사회적 평가를 심각하게 침해했다."고 썼으며, 이 후보는 변 대표를 상대로 1억원 상당의 손해배상청구 소장도 수원지법 성남지원에 제출했다.\n\n변희재에 대한 뉴스를 봤는데, 이재명 시장이 변희재 대표를 고발하여 법적 조치를 취하고, 민사 손해배상소송도 제기했더라고요. 이게 무슨 일인가요? ASSISTANT: 이재명 시장이 변희재 대표가 자신의 SNS에서 명예훼손, 모욕, 공직선거법 위반 등의 혐의로 이재명 시장을 고소한 것에 대해 반격을 해서 법적 조치와 손해배상 청구까지 이어졌다는 거죠.\n\nUSER: 변희재 대표가 이재명 시장을 명예훼손하거나 모욕한 사실이 있는 건가요? ASSISTANT: 그게 정확히 무슨 내용인지는 모르지만, 변희재 대표가 이재명 시장을 \'종북세력\'으로 지칭하고 \'북한의 지령을 받고 움직이는 자들은 종북이 아니라 간첩\'이라는 내용을 SNS에 총 13차례에 걸쳐 올려서 이재명 시장이 본인의 명예를 훼손하였다는 것이죠.\n\n'

In [35]:
import json

with open("../data/inst_edu_v4.json", "r") as fin:
    data = json.load(fin)

with open("../data/processed_math_eng_data.json", "r") as fin:
    add_data = json.load(fin)

data.extend(to_dialog(add_data, "user", "response", "education"))
len(data)

100%|██████████| 4418/4418 [00:00<00:00, 121457.69it/s]

Get 4375 dialogs


77880

In [1]:
import json
with open("../data/inst_edu_v4.json", "r") as fout:
    data = json.load(fout)

In [2]:
import sys
sys.path.append("..")
from model_utils.conversation import get_conv_template

conv = get_conv_template("elementgpt_for_teacher")
roles = {"human": conv.roles[0], "bot": conv.roles[1]}

# Apply prompt templates
conversations = []
for i, source in enumerate(data[-2000:-1998]):
        
    if roles[source[0]["from"]] != conv.roles[0]:
        # Skip the first one if it is not from human
        source = source[1:]

    conv.messages = []
    for j, sentence in enumerate(source):
        role = roles[sentence["from"]]
        assert role == conv.roles[j % 2], f"{i}"
        conv.append_message(role, sentence["value"])
    
    conversations.append(conv.get_prompt())

conversations

['초등학교 교사와 인공지능 어시스턴트 간의 대화. 어시스턴트는 교육에 대한 유저의 질문이나 지시에 도움이 되는 답변을 합니다.\n\n### 질문: 1학년 2학기 수학과목 시계 보기에 관한 단답형 문제 네 문제 만들어줘.\n\n### 답변: 문제1: 시계의 짧은바늘이 9와 10의 가운데를, 긴바늘이 6을 가리키고 있습니다. 시계가 나타내는 시각을 쓰시오.  \n정답 및 해설: 시계의 긴바늘이 6 을 가리키면 30 분이고, 짧은바늘이 9 와 10 의 가운데를 가리키므로 9 시 30 분입니다. \n\n문제2: 민규는 아래 시각부터 방과 후 수업을 시작합니다. 시계의 긴바늘이 3바퀴 돌았을 때, 짧은바늘이 가리키는 곳은 어디입니까?\n정답 및 해설: 시계의 긴바늘이 한 바퀴를 돌 때 짧은 바늘은 한 칸씩 움직이므로 긴바늘이 3 바퀴를 돌면 짧은 바늘은 3 칸을 움직여 3 과 4 의 가운데를 가리키게 됩니다.\n\n문제3: 시계의 긴바늘과 짧은바늘이 모두 숫자 12를 가리키고 있습니다. 시계가 나타내는 시각을 구하시오. 시\n정답 및 해설: 시계의 긴바늘이 숫자 12 를 가리키면 ‘몇 시’인데 짧은바늘이 숫자 12 를 가리키므로 12 시입니다. 따라서 시계가 나타내는 시각은 12 시입니다.\n\n문제4: 정연이는 3시에 학원에 가서 5시에 집에 돌아온다고 합니다. 정연이가 학원에 가서 집에 돌아오는 시간 동안 시계의 긴바늘은 몇 바퀴 도는지 구하시오. 바퀴\n정답 및 해설: 따라서 3 시에서 5 시가 되려면 시계의 긴바늘이 2 바퀴 돕니다. 시계의 긴바늘이 가리키는 숫자가 그대로일 때 짧은바늘이 가리키는 숫자가 1 커지면 긴바늘이 1 바퀴 돈 것입니다.<|endoftext|>',
 '초등학교 교사와 인공지능 어시스턴트 간의 대화. 어시스턴트는 교육에 대한 유저의 질문이나 지시에 도움이 되는 답변을 합니다.\n\n### 질문: 1학년 2학기 수학과목 시계 보기에 관한 단답형 문제 3 문항 만들어줘.\n\n### 답변: 문제1: 상민이는 5시에 피아노 학원에 가고, 친구

In [4]:
import torch
data_dict = torch.load("../data/inst_edu_v4.pt")            
input_ids = data_dict["input_ids"]
labels = data_dict["labels"]
input_ids[0]

tensor([ 7289,   647,   750,   296, 12917,   441,  5626,   501,  6107, 16940,
          965,   285,  2531,    17,   501,  6107, 16940,   272, 12917,   285,
         2438,  1019,  3634,   274,  1980,   270,   740,   283,  9499,   284,
          453, 18014,   288,  4253,   276,  2802,    17,   202,   202,     6,
            6,     6,  2438,    29, 11535,   272,  1382,  6587,  6548, 11260,
           34,  1273,   543,   425,  9751,  4408, 11260,    34,   202,   202,
            6,     6,     6,  4253,    29, 11535,   272,  9964,   270,  1577,
         6587,   285,  8260,  1649,  1441,    17,   543,   425,  9751,  6587,
          285,  4408,  1649,  1441,    17,   224,   202,   202,  9600,   285,
         6548,   285,  4878,   274,   879,  3716, 20739,   272,   669,   270,
         2016,  2670,   307,  2438,   274,   741,  1865,   276,  1044,   283,
          327,   276,   388,  1441,    17, 11535,   272,  9964,   270,  1577,
         8260,  1649,  1441,    17,   543,   425,  9751,  1108, 